# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,9.22,87,100,11.37,IS,1731361025
1,1,avarua,-21.2078,-159.7750,25.03,65,75,6.17,CK,1731361027
2,2,thompson,55.7435,-97.8558,-3.91,86,100,1.54,CA,1731361021
3,3,gadzhiyevo,69.2551,33.3362,2.24,87,100,7.39,RU,1731361029
4,4,manado,1.4870,124.8455,23.43,100,20,0.51,ID,1731361031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing cities from city_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude"
)
# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 27
wind_speed_s = city_data_df["Wind Speed"] < 4.5
cloud_max = city_data_df["Cloudiness"] == 0

print(sum(min_temp_s))
print(sum(max_temp_s))
print(sum(wind_speed_s))
print(sum(cloud_max))

222
474
389
544


In [12]:

# Drop any rows with null values
city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]
city_temp_range_df

# Display sample data
clean_city_temp_range_df = city_temp_range_df.dropna()
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,manado,1.4870,124.8455,23.43,100,0,0.51,ID,1731361031
14,14,adamstown,-25.0660,-130.1015,21.34,65,0,1.23,PN,1731361044
20,20,quepos,9.4236,-84.1652,23.31,97,0,2.76,CR,1731361055
36,36,makoua,0.0069,15.6333,22.90,85,0,0.87,CG,1731361075
43,43,bardai,11.6500,7.2333,21.17,33,0,3.80,NG,1731360907
...,...,...,...,...,...,...,...,...,...,...
531,531,kikwit,-5.0410,18.8162,21.20,86,0,0.35,CD,1731361943
533,533,hanceville,34.0607,-86.7675,24.53,55,0,2.07,US,1731361945
536,536,port-gentil,-0.7193,8.7815,26.98,89,0,4.12,GA,1731361949
537,537,bobo-dioulasso,11.1771,-4.2979,22.25,60,0,2.57,BF,1731361950


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,manado,ID,1.4870,124.8455,100,
14,adamstown,PN,-25.0660,-130.1015,65,
20,quepos,CR,9.4236,-84.1652,97,
36,makoua,CG,0.0069,15.6333,85,
43,bardai,NG,11.6500,7.2333,33,
...,...,...,...,...,...,...
531,kikwit,CD,-5.0410,18.8162,86,
533,hanceville,US,34.0607,-86.7675,55,
536,port-gentil,GA,-0.7193,8.7815,89,
537,bobo-dioulasso,BF,11.1771,-4.2979,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
manado - nearest hotel: Swiss Belhotel
adamstown - nearest hotel: No hotel found
quepos - nearest hotel: Casa Feliz
makoua - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
moissala - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
pitimbu - nearest hotel: Pousada Brisa Mar
holualoa - nearest hotel: Kona Hotel
siguiri - nearest hotel: No hotel found
salahonda - nearest hotel: No hotel found
mossendjo - nearest hotel: Louesse Palace
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
dompu - nearest hotel: No hotel found
karachi - nearest hotel: Quetta Agha Hotel
east ballina - nearest hotel: Ballina Beach Resort
sibut - nearest hotel: No hotel found
aklera - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
axim - nearest hotel: Axim Beach Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
georgetown - nearest hotel: Page 63 h

,City,Country,Lat,Lng,Humidity,Hotel Name
4,manado,ID,1.4870,124.8455,100,Swiss Belhotel
14,adamstown,PN,-25.0660,-130.1015,65,No hotel found
20,quepos,CR,9.4236,-84.1652,97,Casa Feliz
36,makoua,CG,0.0069,15.6333,85,No hotel found
43,bardai,NG,11.6500,7.2333,33,No hotel found
...,...,...,...,...,...,...
531,kikwit,CD,-5.0410,18.8162,86,Grand Hôtel de Kikwit
533,hanceville,US,34.0607,-86.7675,55,No hotel found
536,port-gentil,GA,-0.7193,8.7815,89,Tara-Mé
537,bobo-dioulasso,BF,11.1771,-4.2979,60,Teria


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2.0,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing ideal cities for vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)